In [ ]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import spacy  # Import spaCy
import re

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# Set the path to Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the video
cap = cv2.VideoCapture('video.mp4')

# Define rectangle coordinates and dimensions
box_x = 0
box_y = 130
box_width = 860
box_height = 85

red_color = (0, 0, 255)
data = {'personName': [], 'message': []}

frame_number = 660
person1 = 'Dad'
person2 = ''

while True:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        break

    t = box_y
    personName = ''
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.destroyAllWindows()
    
    while True:
        if t > 900:
            cv2.destroyAllWindows()
            break
            
        grayImage_region_req = grayImage[t:t+box_height, box_x:box_x+box_width]
        
        chattingWith = grayImage[65:115, 390:470]
        person2 = pytesseract.image_to_string(chattingWith, lang='eng')
        person2 = ''.join(person2.splitlines())

            
#         cv2.imshow('adad', grayImage_region_req)
#         cv2.waitKey(0)
        text = pytesseract.image_to_string(grayImage_region_req, lang='eng')
        
        if (np.mean(grayImage_region_req) < 228):
            if personName == 'son':
                t = t+2
            personName = person1
        else:
            if personName == 'Dad':
                t = t+2
            personName = person2

        if text.strip():  # Check if the text is not empty or just whitespace
            # Remove noise words using regular expressions
            text = re.sub(r'\b(Delivered|iMessage Today \d+:\d+[apm]{2})\b', '', text)
            text = ''.join(text.splitlines())
            data['personName'].append(personName)
            data['message'].append(text.strip())  # Remove leading and trailing whitespace
        print(f"Person: {personName}, Text: {text}")

        t = t+box_height+2
    frame_number = frame_number + 660

# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)

# Choose a file path for the Excel file
excel_file_path = 'output_data.xlsx'

# Save the data to an Excel file
df.to_excel(excel_file_path, index=False)
